In [58]:
%load_ext jupyter_black

In [63]:
import gzip
import shutil
from pathlib import Path
from typing import Iterable
from datetime import datetime, timedelta
from urllib3.response import HTTPResponse

import pandas as pd
import xarray as xr
import rasterio as rio
from requests import Session


def make_urls(start: datetime, stop: int, step: int = 0, item="RadarOnly_QPE_01H_00.00") -> Iterable[str]:
    # https://mtarchive.geol.iastate.edu/2022/07/15/mrms/ncep/RadarOnly_QPE_01H/RadarOnly_QPE_01H_00.00_20220715-000000.grib2.gz
    end = start + timedelta(hours=stop)
    yield from (
        "https://mtarchive.geol.iastate.edu/"
        + pd.date_range(start, end, freq="2h").strftime(
            "%Y/%m/%d/mrms/ncep/RadarOnly_QPE_01H/RadarOnly_QPE_01H_00.00_%Y%m%d-00%H%M"
        )
        + ".grib2.gz"
    )


def gzip_request(file_path: Path, raw: HTTPResponse) -> None:
    with gzip.open(raw, mode="rb") as f_in:
        with file_path.open(mode="wb") as f_out:
            shutil.copyfileobj(f_in, f_out)


def mrms_dataset_from_urls(urls: Iterable[str], path: Path = Path("/tmp/dataset")) -> xr.Dataset:
    if not path.exists():
        path.mkdir()

    def generate():
        with Session() as session:
            for i, url in enumerate(urls):
                file_name = path / f"outfile-{i}.grib2"
                res = session.get(url, stream=True)
                res.raise_for_status()
                gzip_request(file_name, res.raw)
                yield file_name

    data = xr.open_mfdataset(
        generate(),
        engine="cfgrib",
        combine="nested",
        concat_dim="valid_time",
    )
    shutil.rmtree(path)

    return data


if __name__ == "__main__":
    urls = make_urls(datetime(2022, 6, 15, 0), 3)
    ds = mrms_dataset_from_urls(urls)
ds

<xarray.Dataset>
Dimensions:         (valid_time: 2, latitude: 3500, longitude: 7000)
Coordinates:
    time            (valid_time) datetime64[ns] 2022-06-15 2022-06-15T00:02:00
    step            timedelta64[ns] 00:00:00
    heightAboveSea  float64 0.0
  * latitude        (latitude) float64 54.99 54.98 54.98 ... 20.03 20.02 20.01
  * longitude       (longitude) float64 230.0 230.0 230.0 ... 300.0 300.0 300.0
  * valid_time      (valid_time) datetime64[ns] 2022-06-15 2022-06-15T00:02:00
Data variables:
    unknown         (valid_time, latitude, longitude) float32 dask.array<chunksize=(1, 3500, 7000), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             161
    GRIB_centreDescription:  161
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             161
    history:                 2022-07-15T22:29 GRIB to CDM+CF via cfgrib-0.9.1...